In [2]:
from datetime import datetime

import numpy as np
import comet_ml
import os
import torch
import torch.optim as optim
import torch.nn as nn

import code_S3R.utils.data_utils
from code_S3R import my_nets
import code_S3R.utils.other_utils as utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
batch_size = 64

In [3]:
# keep quiet, scipy
utils.hide_scipy_zoom_warnings()

# Load the dataset
import torch

class OnlineDHGDataset(torch.utils.data.Dataset):
    def __init__(self, train_set=True):
        x_train, y_train = code_S3R.utils.data_utils.load_dataset_in_torch(use_online_dataset=True, use_14=True,
                                                                           train_dataset=True)
        x_test, y_test = code_S3R.utils.data_utils.load_dataset_in_torch(use_online_dataset=True, use_14=True,
                                                                         train_dataset=False)
        y_train = y_train.type_as(torch.FloatTensor())
        y_test = y_test.type_as(torch.FloatTensor())
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        self.train_set = train_set

    def __len__(self):
        if self.train_set:
            return len(self.x_train)
        else:
            return len(self.x_test)

    def __getitem__(self, index):
        if self.train_set:
            return self.x_train[index], self.y_train[index]
        else:
            return self.x_test[index], self.y_test[index]


training_set = OnlineDHGDataset(train_set=True)
test_set = OnlineDHGDataset(train_set=False)

print(len(training_set))
training_generator = torch.utils.data.DataLoader(training_set, batch_size=batch_size)
testing_generator = torch.utils.data.DataLoader(test_set, batch_size=batch_size)


21491


In [4]:
# define network
net = my_nets.Net(
    preprocess=None,
    conv_type='regular',
    channel_list=[(96, 1)],
    fc_hidden_layers=[1024, 128],
    activation_fct='prelu',
    dropout=0.4,
    temporal_attention=None,
    nb_classes=14,
)

In [5]:
# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [8]:
# train
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(training_generator, 0):
        # get the inputs
        inputs, labels = data
        
        print(inputs.shape)
        print(labels.shape)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

torch.Size([64, 100, 66])
torch.Size([64, 15])


RuntimeError: Expected object of type torch.LongTensor but found type torch.FloatTensor for argument #2 'target'

In [8]:
nb_classes=14

net = NeuralNetClassifier(
    module=my_nets.Net,
    max_epochs=2000, batch_size=128,
    lr=0.0001,
    module__preprocess=None,  # or None
    module__conv_type='regular',
    module__channel_list=[(96, 1)],
    module__fc_hidden_layers=[1024, 128],
    module__activation_fct='prelu',
    module__dropout=0.4,
    module__temporal_attention=None,
    module__nb_classes=nb_classes,

    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    callbacks=[
        #('my_cb', utils.MyCallback(param_keys_to_log=utils.get_param_keys(hyper_params),
        #                           search_run_id=single_run_id,
        #                           log_to_comet_ml=False, log_to_csv=True)),
        ('early_stopping', callbacks.EarlyStopping(patience=50)),
        # ('f1', EpochScoring('f1', name='f1', lower_is_better=False,))
    ],
    device=device,
    train_split=CVSplit(cv=5, stratified=False),
)

In [7]:
net.fit()

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'code_S3R.my_nets.Net'>,
  module__activation_fct=prelu,
  module__channel_list=[(528, 66), (528, 66), (264, 66)],
  module__conv_type=regular,
  module__dropout=0.4,
  module__fc_hidden_layers=[1996],
  module__nb_classes=14,
  module__preprocess=None,
  module__temporal_attention=None,
)

In [1]:
# 95% confiance interval
(0.005220835843908473 *1.96)/3**(1/2)

0.005907931920555792

In [12]:
# format : (N, C, L)
x = torch.randn(8, 14, 10)

x[:,:,-1].shape

torch.Size([8, 14])